### Formatting labels on plots.

In Lets-Plot you can apply a formatting to:

- axis break values.
- labels displayed by `geomText()`.
- tooltip text.
- facet labels.

Using format string you can format values of numeric and date-time types.

In addition, you can use a *string template*.

In *string template* the value's format string is surrounded by curly braces: `"... {.2f} ..."`.

An empty placeholder `{}` is also allowed. In this case a default string representation will be shown. This is also applicable to categorical values.

To learn more about formatting templates see: [Formatting](https://github.com/JetBrains/lets-plot-kotlin/blob/master/docs/formats.md).

In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.4.1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.2.0.

### The US Unemployment Rates 2000-2016

In [3]:
var economics = DataFrame.readCSV("https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/economics.csv")

In [4]:
economics.head()

DataFrame: rowsCount = 5, columnsCount = 7

In [5]:
import kotlinx.datetime.*

fun LocalDate.toEpochMilliseconds(): Long {
    return atStartOfDayIn((TimeZone.currentSystemDefault())).toEpochMilliseconds()
}

In [6]:
val startDate = LocalDate.parse("2000-01-01")
economics = economics.filter { date.compareTo(startDate) >= 0 }.convert {date}.with {it.toEpochMilliseconds()}

In [7]:
val p = letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
    geomLine() +
    ylab("unemployment rate") +
    ggsize(900, 400)

#### Default plot (no formatting)

#### Apply formatting to X and Y axis labels.

Use the `format` parameter in `scaleXxx()`.

Note that the text in tooltips is now also formatted.

In [8]:
p + scaleXDateTime(format="%b %Y") + scaleYContinuous(format="{} %")

#### Format axis labels for breaks specified manually


In [9]:
val breaks = listOf(
    LocalDate.parse("2001-01-01"), 
    LocalDate.parse("2016-01-01")
).map {it.toEpochMilliseconds()}

p + scaleXDateTime(format="%b %Y", breaks=breaks) + scaleYContinuous(format="{} %")

#### Configure tooltip's text and location.

In [10]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y")
        .anchor("top_center")
        .minWidth(170)) + 
 scaleXDateTime(format="%b %Y") +
 scaleYContinuous(format="{} %") +
 ylab("unemployment rate") +
 ggsize(900, 400)

#### Format value shown in tooltip.

In [11]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} +
 geomLine(tooltips=layerTooltips()
               .line("@uempmed % in @date")
               .format("date", "%B %Y")
               .anchor("top_left")
               .minWidth(170)) +
 scaleXDateTime() +
 scaleYContinuous() +
 ylab("unemployment rate") +
 ggsize(900, 400)

#### Add the unemployment rate mean.

The `geomText` label is formatted using the `labelFormat` parameter.

In [12]:
val unemploymentMean = economics.uempmed.mean()


letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y %")
        .anchor("top_center")
        .minWidth(170)) + 
 geomHLine(yintercept=unemploymentMean, color="red", linetype="dashed", tooltips=tooltipsNone) +
 geomText(label=unemploymentMean.toString(), 
           labelFormat="{.2f} %",
           x=startDate.toEpochMilliseconds(), y=unemploymentMean!! + 0.5, 
           color="red") +
 scaleXDateTime(format="%b %Y") + 
 scaleYContinuous(format="{} %") + 
 ylab("unemployment rate") +
 ggtitle("The US Unemployment Rates 2000-2016.") +
 ggsize(900, 400)